##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizado federado para classificação de imagens

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

**OBSERVAÇÃO**: foi verificado que este Colab funciona com a [versão mais recente lançada](https://github.com/tensorflow/federated#compatibility) do pacote pip `tensorflow_federated`, mas o projeto TensorFlow Federated ainda está em desenvolvimento pré-lançamento e pode não funcionar no `main`.

Neste tutorial, usamos o exemplo clássico de treinamento MNIST para apresentar a camada de API do aprendizado federado (FL) do TFF, `tff.learning`: um conjunto de interfaces de alto nível que podem ser usadas para fazer tarefas comuns de aprendizado federado, como treinamento federado, usando modelos fornecidos por usuários e implementados no TensorFlow.

Este tutorial e a API de aprendizado federado visam principalmente usuários que querem conectar seus próprios modelos do TensorFlow ao TFF, tratando o último como se fosse basicamente uma caixa preta. Para uma compreensão mais aprofundada do TFF e de como implementar seus próprios algoritmos de aprendizado federado, consulte os tutoriais na API FC Core: [Algoritmos federados personalizados, parte 1](custom_federated_algorithms_1.ipynb) e [parte 2](custom_federated_algorithms_2.ipynb).

Para saber mais sobre `tff.learning`, continue com o [Aprendizado federado para geração de texto](federated_learning_for_text_generation.ipynb), um tutorial que, além de abordar modelos recorrentes, demonstra como carregar um modelo Keras serializado pré-treinado para que seja refinado com uma combinação de avaliação e aprendizado federado usando o Keras.

## Antes de começarmos

Antes de começarmos, execute o código abaixo para que o ambiente seja configurado corretamente. Se não for exibida uma saudação, consulte as instruções de [instalação](../install.md). 

In [ ]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated

In [ ]:
%load_ext tensorboard

Fetching TensorBoard MPM version 'live'... done.


In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Prepare os dados de entrada

Vamos começar pelos dados. O aprendizado federado requer um conjunto federado de dados, ou seja, uma coleção de dados de diversos usuários. Geralmente, os dados federados não são [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables), o que traz desafios únicos.

Para facilitar a experimentação, propagamos o repositório do TFF com alguns datasets, incluindo uma versão federada do MNIST que contém uma versão do [dataset NIST original](https://www.nist.gov/srd/nist-special-database-19), reprocessado usando [Leaf](https://github.com/TalwalkarLab/leaf) para que os dados sejam digitados pelo escritor original dos dígitos. Como cada escritor tem um estilo único, esse dataset exibe o tipo de comportamento não i.i.d. esperado de datasets federados.

Veja como podemos carregá-lo.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Os datasets retornados por `load_data()` são instâncias de `tff.simulation.ClientData`, uma interface que permite enumerar o conjunto de usuários, para construir um `tf.data.Dataset` que representa os dados de um usuário específico e para consultar a estrutura de elementos individuais. Veja como você pode usar essa interface para explorar o conteúdo do dataset. Lembre-se de que, embora essa interface permita iterar as ids dos clientes, é apenas um recurso dos dados de simulação. Como você verá em breve, as identidades dos clientes não são usadas pelo framework de aprendizado federado — seu único propósito é permitir que você selecione subconjuntos de dados para simulações.

In [ ]:
len(emnist_train.client_ids)

3383

In [ ]:
emnist_train.element_type_structure

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])

In [ ]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_element = next(iter(example_dataset))

example_element['label'].numpy()

1

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

### Explore a heterogeneidade nos dados federados

Normalmente, os dados federados não são [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) e os usuários têm distribuições de dados diferentes dependendo dos padrões de uso. Alguns clientes podem ter menos exemplos de treinamento no dispositivo, sofrendo com a escassez de dados localmente, e outros terem exemplos de treinamento mais do que suficientes. Vamos explorar esse conceito de heterogeneidade de dados típico de um sistema federado com os dados EMNIST que temos disponíveis. É importante observar que esta análise profunda dos dados de um cliente só está disponível para nós porque é um ambiente de simulação onde todos os dados estão disponíveis localmente. Em um ambiente federado de produção real, não seria possível inspecionar os dados de um único cliente.

Primeiro, vamos pegar uma amostra dos dados de um cliente para ter uma ideia dos exemplos em um dispositivo simulado. Como o dataset que estamos usando foi digitado por um único escritor, os dados de um cliente representam a caligrafia de uma pessoa para uma amostra dos dígitos 0 a 9, simulando o "padrão de uso" exclusivo de um usuário.

In [ ]:
## Example MNIST digits for one client
figure = plt.figure(figsize=(20, 4))
j = 0

for example in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(example['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

Agora, vamos visualizar o número de exemplos em cada cliente para cada rótulo de dígito MNIST. No ambiente federado, o número de exemplos em cada cliente pode variar bastante, dependendo do comportamento do usuário.

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12, 7))
f.suptitle('Label Counts for a Sample of Clients')
for i in range(6):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    # Append counts individually per label to make plots
    # more colorful instead of one color per plot.
    label = example['label'].numpy()
    plot_data[label].append(label)
  plt.subplot(2, 3, i+1)
  plt.title('Client {}'.format(i))
  for j in range(10):
    plt.hist(
        plot_data[j],
        density=False,
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Vamos visualizar a imagem média por cliente para cada rótulo MNIST. Esse código produzirá a média de cada valor de pixel para todos os exemplos do usuário para um rótulo. Veremos que a imagem média de um cliente para um dígito será completamente diferente da imagem média de outro cliente para o mesmo dígito, devido ao estilo de caligrafia único de cada pessoa. Podemos refletir sobre como cada rodada de treinamento local leva o modelo em uma direção diferente para cada cliente, conforme aprendemos com os dados únicos desse usuário na rodada local. Mais adiante no tutorial, veremos como pegar cada atualização do modelo de todos os clientes e agregá-las em nosso novo modelo global, que aprendeu com os dados únicos de cada um dos clientes.

In [ ]:
# Each client has different mean images, meaning each client will be nudging
# the model in their own directions locally.

for i in range(5):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    plot_data[example['label'].numpy()].append(example['pixels'].numpy())
  f = plt.figure(i, figsize=(12, 5))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(10):
    mean_img = np.mean(plot_data[j], 0)
    plt.subplot(2, 5, j+1)
    plt.imshow(mean_img.reshape((28, 28)))
    plt.axis('off')

Os dados do usuário podem ser ruidosos e rotulados de maneira pouco confiável. Por exemplo, observando os dados do cliente nº 2 acima, para o rótulo 2, alguns exemplos podem ter sido mal rotulados, criando uma imagem média mais ruidosa.

### Pré-processe os dados de entrada

Como os dados já são um `tf.data.Dataset`, o pré-processamento pode ser realizado usando transformações de dataset. Achatamos as imagens `28x28` em arrays de `784` elementos, embaralhamos os exemplos individuais, organizamos-os em lotes e renomeamos os recursos de `pixels` e `label` para `x` e `y` a fim de usá-los com o Keras. Também incluímos `repeat` no dataset para executar várias épocas.

In [ ]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

Vamos verificar se funcionou.

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

OrderedDict([('x', array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)), ('y', array([[2],
       [1],
       [5],
       [7],
       [1],
       [7],
       [7],
       [1],
       [4],
       [7],
       [4],
       [2],
       [2],
       [5],
       [4],
       [1],
       [1],
       [0],
       [0],
       [9]], dtype=int32))])

Temos quase todos os blocos básicos para construir datasets federados.

Uma das formas de alimentar o TFF com dados federados em uma simulação é simplesmente como uma lista Python, com cada elemento da lista contendo os dados de um usuário individual, seja como uma lista ou como um `tf.data.Dataset`. Já temos uma interface que oferece este último, então vamos usá-la.

Veja abaixo uma função helper simples que construirá uma lista de datasets a partir do conjunto fornecido de usuários como uma entrada para uma rodada de treinamento ou avaliação.

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

Agora, como escolhemos os clientes?

Em um cenário de treinamento federado típico, lidamos com uma população possivelmente muito grande de dispositivos de usuários, sendo que apenas uma fração deles pode estar disponível para treinamento em um determinado momento. Este é o caso, por exemplo, quando os dispositivos dos clientes são celulares que participam do treinamento apenas quando conectados a uma fonte de energia, fora de uma rede limitada e, de outra forma, inativos.

É claro que estamos em um ambiente de simulação, e todos os dados estão disponíveis localmente. Normalmente, ao executar simulações, simplesmente tomaríamos como amostra um subconjunto aleatório de clientes envolvidos em cada rodada de treinamento, geralmente diferentes a cada rodada.

Dito isso, como você pode descobrir estudando o artigo sobre o algoritmo de [cálculo federado de médias](https://arxiv.org/abs/1602.05629), alcançar a convergência em um sistema com subconjuntos de clientes amostrados aleatoriamente em cada rodada pode demorar um pouco, e seria impraticável executar centenas de rodadas neste tutorial interativo.

Em vez disso, o que vamos fazer é pegar uma única amostra de conjunto de clientes e reutilizar o mesmo conjunto em todas as rodadas para acelerar a convergência (fazendo o overfitting intencional dos dados desses poucos usuários). Deixamos como um exercício para o leitor modificar este tutorial para simular uma amostragem aleatória — é bastante fácil (depois de fazer isso, lembre-se de que a convergência do modelo pode demorar).

In [ ]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(emnist_train, sample_clients)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Number of client datasets: 10
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>


## Crie um modelo com o Keras

Se você estiver usando o Keras, provavelmente já tem algum código que construa um modelo do Keras. Veja abaixo um exemplo de um modelo simples que será suficiente para nossas necessidades.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

**Observação:** ainda não compilamos o modelo. A perda, as métricas e os otimizadores serão apresentados mais tarde.

Para usar qualquer modelo com o TFF, ele precisa ser envolvido em uma instância da interface `tff.learning.models.VariableModel`, que expõe métodos para carimbar o passo para frente do modelo, as propriedades de metadados etc., semelhante ao Keras, mas também introduz elementos adicionais, como formas de controlar o processo de computação das métricas federadas. Não vamos nos preocupar com isso por enquanto. Se você tiver um modelo do Keras como o que acabamos de definir acima, poderá fazer com que o TFF o envolva invocando `tff.learning.models.from_keras_model`, passando o modelo e um lote de dados de amostra como argumentos, conforme mostrado abaixo.

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Treine o modelo com dados federados

Agora que temos um modelo envolvido como `tff.learning.models.VariableModel` para uso com o TFF, podemos permitir que o TFF construa um algoritmo de cálculo federado de média invocando a função helper `tff.learning.algorithms. build_weighted_fed_avg` da maneira a seguir.

Tenha em mente que o argumento precisa ser um construtor (como `model_fn` acima), e não uma instância já construída, para que a construção do modelo possa acontecer em um contexto controlado pelo TFF (se você quiser saber os motivos disso, recomendamos ler o tutorial seguinte sobre [algoritmos personalizados](custom_federated_algorithms_1.ipynb)).

Uma observação importante sobre o cálculo federado de média abaixo — há **2** otimizadores: um *client_optimizer* e um *server_optimizer*. O *client_optimizer* é usado apenas para calcular atualizações do modelo local em cada cliente. O *server_optimizer* aplica a atualização média ao modelo global no servidor. Especificamente, isso significa que talvez seja necessário escolher outro otimizador e taxa de aprendizado em vez dos usados para treinar o modelo com um dataset i.i.d padrão. Recomendamos começar com um SGD regular, possivelmente com uma taxa de aprendizado menor do que o normal. A taxa de aprendizado que usamos não foi cuidadosamente ajustada, então sinta-se à vontade para experimentar.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

O que acabou de acontecer? O TFF construiu um par de *computações federadas* e as empacotou em um `tff.templates.IterativeProcess` onde essas computações estão disponíveis como um par de propriedades `initialize` e `next`.

Resumindo, as *computações federadas* são programas na linguagem interna do TFF que podem expressar vários algoritmos federados (saiba mais sobre isso no tutorial sobre [algoritmos personalizados](custom_federated_algorithms_1.ipynb)). Nesse caso, as duas computações geradas e compactadas em `iterative_process` implementam o [cálculo federado de média](https://arxiv.org/abs/1602.05629).

É um objetivo do TFF definir computações de modo que possam ser executadas em ambientes reais de aprendizado federado. Porém, no momento, apenas o runtime de simulação da execução local foi implementado. Para executar uma computação em um simulador, basta invocá-la como uma função Python. Esse ambiente interpretado padrão não foi projetado para alto desempenho, mas será suficiente para este tutorial. Esperamos fornecer runtimes de simulação de alto desempenho para facilitar pesquisas em maior escala em versões futuras.

Vamos começar com a computação `initialize`. Como acontece com todas as computações federadas, você pode pensar nela como uma função. A computação não aceita argumentos e retorna um resultado — a representação do estado do processo de cálculo federado de média no servidor. Não queremos nos aprofundar nos detalhes do TFF, mas pode ser esclarecedor ver como é esse estado. Você pode visualizá-lo da seguinte maneira.

In [ ]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[784,10],
      float32[10]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[784,10],
    float32[10]
  >
>@SERVER)


A assinatura de tipo acima pode parecer um pouco obscura no início, mas é possível reconhecer que o estado do servidor consiste em um `global_model_weights` (os parâmetros iniciais do modelo para MNIST que serão distribuídos para todos os dispositivos), alguns parâmetros vazios (como `distributor`, que controla a comunicação servidor-cliente) e um componente `finalizer`. Este último rege a lógica que o servidor usa para atualizar seu modelo ao final de uma rodada e contém um número inteiro que representa quantas rodadas de FedAvg foram realizadas.

Vamos invocar a computação `initialize` para construir o estado do servidor.

In [ ]:
train_state = training_process.initialize()

O segundo item do par de computações federadas, `next`, representa uma única rodada de cálculo federado de média, que consiste em enviar o estado do servidor (incluindo os parâmetros do modelo) para os clientes, o treinamento no dispositivo usando os dados locais, a coleta e média das atualizações do modelo e a geração de um modelo atualizado no servidor.

Conceitualmente, você pode pensar que `next` tem uma assinatura de tipo funcional semelhante a esta.

```
SERVER_STATE, FEDERATED_DATA -&gt; SERVER_STATE, TRAINING_METRICS
```

Em particular, não se deve pensar em `next()` como uma função executada em um servidor, e sim como uma representação funcional declarativa de toda a computação descentralizada. Algumas das entradas são fornecidas pelo servidor (`SERVER_STATE`), mas cada dispositivo participante contribui com seu próprio dataset local.

Vamos executar uma única rodada de treinamento e ver os resultados. Podemos usar os dados federados já gerados acima para uma amostra de usuários.

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.12345679), ('loss', 3.1193733), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


Vamos executar mais algumas rodadas. Conforme dito antes, geralmente, a esta altura, você deve escolher um subconjunto dos seus dados de simulação usando uma nova amostra de usuários selecionada aleatoriamente para cada rodada. O intuito disso é simular uma implantação realista, em que os usuários chegam e saem o tempo todo. Porém, neste notebook interativo, para fins de demonstração, vamos reutilizar os mesmos usuários para que a convergência do sistema seja rápida.

In [ ]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.14012346), ('loss', 2.9851403), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.1590535), ('loss', 2.8617127), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.17860082), ('loss', 2.7401376), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('fin

A perda de treinamento está diminuindo a cada rodada de treinamento federado, indicando que o modelo está convergindo. Há algumas ressalvas importantes sobre essas métricas de treinamento, que são explicadas mais adiante, na seção *Avaliação* deste tutorial.

## Exiba métricas do modelo no TensorBoard

Agora, vamos visualizar as métricas dessas computações federadas usando o Tensorboard.

Vamos começar criando o diretório e o escritor de resumo correspondente para a escrita das métricas.


In [ ]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
try:
  tf.io.gfile.rmtree(logdir)  # delete any previous results
except tf.errors.NotFoundError as e:
  pass # Ignore if the directory didn't previously exist.
summary_writer = tf.summary.create_file_writer(logdir)
train_state = training_process.initialize()

Plote as métricas de escalares relevantes com o mesmo escritor de resumo.

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    result = training_process.next(train_state, federated_train_data)
    train_state = result.state
    train_metrics = result.metrics
    for name, value in train_metrics['client_work']['train'].items():
      tf.summary.scalar(name, value, step=round_num)

Inicialize o TensorBoard com o diretório de log raiz especificado acima. O carregamento dos dados pode demorar alguns segundos.

In [ ]:
#@test {"skip": true}
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [ ]:
#@test {"skip": true}
# Uncomment and run this cell to clean your directory of old output for
# future graphs from this directory. We don't run it by default so that if 
# you do a "Runtime > Run all" you don't lose your results.

# !rm -R /tmp/logs/scalars/*

Para ver as métricas de avaliação da mesma forma, você pode criar uma pasta eval separada, como "logs/scalars/eval", para gravar no TensorBoard.

## Personalize a implementação do modelo

O Keras é a [API de modelo de alto nível recomendada para o TensorFlow](https://medium.com/tensorflow/standardizing-on-keras-guidance-on-high-level-apis-in-tensorflow-2-0-bad2b04c819a), e recomendamos o uso de modelos do Keras (via `tff.learning.models.from_keras_model`) no TFF sempre que possível.

No entanto, `tff.learning` fornece uma interface de modelo de nível inferior, `tff.learning.models.VariableModel`, que expõe a funcionalidade mínima necessária para usar um modelo no aprendizado federado. A implementação direta dessa interface (possivelmente ainda usando blocos básicos como `tf.keras.layers`) permite a personalização máxima sem modificar os componentes internos dos algoritmos de aprendizado federado.

Vamos fazer tudo de novo do zero.

### Defina variáveis do modelo, passo para frente e métricas

A primeira etapa é identificar as variáveis do TensorFlow com que vamos trabalhar. Para melhorar a legibilidade do código a seguir, vamos definir uma estrutura de dados para representar todo o conjunto. Isso incluirá variáveis como `weights` e `bias` que treinaremos, bem como variáveis com diversas estatísticas cumulativas e contadores que atualizaremos durante o treinamento, como `loss_sum`, `accuracy_sum` e `num_examples`.

In [ ]:
MnistVariables = collections.namedtuple(
    'MnistVariables', 'weights bias num_examples loss_sum accuracy_sum')

Veja a seguir um método para criar as variáveis. Para simplificar, representamos todas as estatísticas como `tf.float32`, pois isso eliminará a necessidade de converter tipos posteriormente. Envolver inicializadores de variáveis como lambdas é um requisito imposto pelas [variáveis de recursos](https://www.tensorflow.org/api_docs/python/tf/enable_resource_variables).

In [ ]:
def create_mnist_variables():
  return MnistVariables(
      weights=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(784, 10)),
          name='weights',
          trainable=True),
      bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(10)),
          name='bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False))

Depois de implementar as variáveis dos parâmetros do modelo e as estatísticas cumulativas, podemos agora definir o método de passo para frente que calcula a perda, emite previsões e atualiza as estatísticas cumulativas para um único lote de dados de entrada, conforme a seguir.

In [ ]:
def predict_on_batch(variables, x):
  return tf.nn.softmax(tf.matmul(x, variables.weights) + variables.bias)

def mnist_forward_pass(variables, batch):
  y = predict_on_batch(variables, batch['x'])
  predictions = tf.cast(tf.argmax(y, 1), tf.int32)

  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y), axis=[1]))
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))

  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  return loss, predictions

Em seguida, definimos duas funções relacionadas às métricas locais, novamente usando o TensorFlow.

A primeira função `get_local_unfinalized_metrics` retorna os valores de métricas não finalizados (além das atualizações do modelo, que são processadas automaticamente) que se qualificam para a agregação no servidor em um processo de avaliação ou aprendizado federado.

In [ ]:
def get_local_unfinalized_metrics(variables):
  return collections.OrderedDict(
      num_examples=[variables.num_examples],
      loss=[variables.loss_sum, variables.num_examples],
      accuracy=[variables.accuracy_sum, variables.num_examples])

A segunda função `get_metric_finalizers` retorna um `OrderedDict` de `tf.function`s com as mesmas chaves (ou seja, nomes de métricas) que `get_local_unfinalized_metrics`. Cada `tf.function` recebe os valores não finalizados da métrica e calcula a métrica finalizada.

In [ ]:
def get_metric_finalizers():
  return collections.OrderedDict(
      num_examples=tf.function(func=lambda x: x[0]),
      loss=tf.function(func=lambda x: x[0] / x[1]),
      accuracy=tf.function(func=lambda x: x[0] / x[1]))

A forma como as métricas locais não finalizadas retornadas por `get_local_unfinalized_metrics` são agregadas em todos os clientes é especificada pelo parâmetro `metrics_gregator` ao definir os processos de avaliação ou aprendizado federado. Por exemplo, na API [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) (mostrada na próxima seção), o valor padrão para `metrics_gregor` é [`tff.learning.metrics.sum_then_finalize`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/metrics/sum_then_finalize), que primeiro soma as métricas não finalizadas de `CLIENTS` e depois aplica os finalizadores de métrica em `SERVER`.

### Construa uma instância de `tff.learning.models.VariableModel`

Depois de implementar todos os componentes acima, podemos construir uma representação de modelo para uso com o TFF semelhante à gerada quando você deixa o TFF ingerir um modelo do Keras.

In [ ]:
import collections
from collections.abc import Callable

class MnistModel(tff.learning.models.VariableModel):

  def __init__(self):
    self._variables = create_mnist_variables()

  @property
  def trainable_variables(self):
    return [self._variables.weights, self._variables.bias]

  @property
  def non_trainable_variables(self):
    return []

  @property
  def local_variables(self):
    return [
        self._variables.num_examples, self._variables.loss_sum,
        self._variables.accuracy_sum
    ]

  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 784], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32))

  @tf.function
  def predict_on_batch(self, x, training=True):
    del training
    return predict_on_batch(self._variables, x)
    
  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    num_exmaples = tf.shape(batch['x'])[0]
    return tff.learning.models.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_exmaples)

  @tf.function
  def report_local_unfinalized_metrics(
      self) -> collections.OrderedDict[str, list[tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to unfinalized values."""
    return get_local_unfinalized_metrics(self._variables)

  def metric_finalizers(
      self) -> collections.OrderedDict[str, Callable[[list[tf.Tensor]], tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to finalizers."""
    return get_metric_finalizers()

  @tf.function
  def reset_metrics(self):
    """Resets metrics variables to initial value."""
    for var in self.local_variables:
      var.assign(tf.zeros_like(var))

Como você pode ver, os métodos e propriedades abstratos definidos por `tff.learning.models.VariableModel` correspondem aos fragmentos de código da seção anterior que apresentaram as variáveis e definiram a perda e as estatísticas.

Alguns pontos que merecem destaque:

- Todos os estados que seu modelo usará precisam ser capturados como variáveis do TensorFlow, já que o TFF não usa o Python no runtime (lembre-se de que o código deve ser escrito de modo que possa ser implantado em dispositivos móveis; consulte o tutorial sobre [algoritmos personalizados](custom_federated_algorithms_1.ipynb) para entender melhor os motivos).
- Seu modelo deve descrever o formato de dados que ele aceita (`input_spec`), já que, em geral, o TFF é um ambiente fortemente tipado e quer determinar as assinaturas de tipo para todos os componentes. Declarar o formato da entrada do modelo é uma parte essencial dele.
- Embora tecnicamente não seja obrigatório, recomendamos agrupar toda a lógica do TensorFlow (passo para frente, cálculos de métricas etc.) como `tf.function`s, porque ajuda a garantir que o TensorFlow possa ser serializado e elimina a necessidade de dependências de controle explícitas.


O exposto acima é suficiente para a avaliação e algoritmos como o Federated SGD. No entanto, para o cálculo federado de média, precisamos especificar como o modelo deve treinar localmente em cada lote. Vamos especificar um otimizador local ao criar o algoritmo de cálculo federado de média.

### Simule o treinamento federado com o novo modelo

Depois de implementar tudo isso, o resto do processo é parecido com o que já vimos — basta substituir o construtor do modelo pelo construtor da nova classe de modelo e usar as duas computações federadas no processo iterativo que você criou para realizar as rodadas de treinamento.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02))

In [ ]:
train_state = training_process.initialize()

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 3.119374), ('accuracy', 0.12345679)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [ ]:
for round_num in range(2, 11):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.98514), ('accuracy', 0.14012346)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.8617127), ('accuracy', 0.1590535)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.740137), ('accuracy', 0.17860082)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  5, metrics=OrderedDict([('distributor', ()), ('client_work', 

Para ver essas métricas no TensorBoard, confira as etapas listadas acima em "Exiba métricas de modelo no TensorBoard".

## Avaliação

Todos os experimentos até agora só apresentaram métricas de treinamento federado — as métricas de média de todos os lotes de dados treinados em todos os clientes na rodada. Isso introduz as preocupações normais sobre overfitting, especialmente porque usamos o mesmo conjunto de clientes em cada rodada para simplificar, mas há uma noção adicional de overfitting nas métricas de treinamento específicas do algoritmo de cálculo federado de média. Isso é mais fácil de ver se imaginarmos que cada cliente tinha um único lote de dados e treinarmos nesse lote por muitas iterações (épocas). Nesse caso, o modelo local se ajustará exatamente a esse lote com rapidez e, por isso, a métrica de precisão local que calculamos a média se aproximará de 1.0. Assim, essas métricas de treinamento podem indicar que ele está progredindo, mas não muito mais que isso.

Para realizar a avaliação em dados federados, você pode construir outra *computação federada* criada exatamente para essa finalidade, usando a função `tff.learning.build_federated_evaluation` e passando o construtor do modelo como um argumento. Observe que, ao contrário do cálculo federado de média, onde usamos `MnistTrainableModel`, basta transmitir o `MnistModel`. A avaliação não realiza o método do gradiente descendente, e não há necessidade de construir otimizadores.

Para experimentação e pesquisa, quando um dataset de teste centralizado está disponível, o [Aprendizado federado para geração de texto](federated_learning_for_text_generation.ipynb) demonstra outra opção de avaliação: pegar os pesos treinados do aprendizado federado, aplicá-los a um modelo Keras padrão e simplesmente chamar `tf.keras.models.Model.evaluate()` em um dataset centralizado.

In [ ]:
evaluation_process = tff.learning.algorithms.build_fed_eval(MnistModel)

Você pode inspecionar a assinatura do tipo abstrato da função de avaliação da seguinte maneira.

In [ ]:
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    x=float32[?,784],
    y=int32[?,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_

Esteja ciente de que o processo de avaliação é um objeto `tff.lenaring.templates.LearningProcess`. O objeto tem um método `initialize` que cria o estado, mas inicialmente contém um modelo não treinado. Usando o método `set_model_weights`, é necessário inserir os pesos do estado de treinamento que será avaliado.

In [ ]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

Agora que o estado de avaliação contém os pesos do modelo que serão avaliados, podemos calcular métricas de avaliação usando datasets de avaliação ao chamar o método `next` no processo, assim como no treinamento.

Novamente, isso retornará uma instância `tff.learning.templates.LearingProcessOutput`.

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_train_data)

Isso é o que obtemos. Observe que os números parecem um pouco melhores do que os relatados na última rodada de treinamento acima. Por convenção, as métricas de treinamento relatadas pelo processo de treinamento iterativo geralmente refletem o desempenho do modelo no início da rodada de treinamento, então as métricas de avaliação estarão sempre um passo à frente.

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)])), ('total_rounds_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

Agora, vamos compilar uma amostra de teste dos dados federados e executar a avaliação novamente para os dados de teste, que virão da mesma amostra de usuários reais, mas de um dataset externo distinto.

In [ ]:
federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

(10,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>)

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)])), ('total_rounds_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

Isso conclui o tutorial. Recomendamos que você brinque com os parâmetros (por exemplo, tamanhos de lote, número de usuários, épocas, taxas de aprendizagem etc.), modifique o código acima para simular o treinamento em amostras aleatórias de usuários em cada rodada e explore os outros tutoriais que nós desenvolvemos.